In [1]:
import toml
import torch
import random
import lightning.pytorch as pl
from pathlib import Path
from aging.size_norm import models
from toolz import dissoc, merge, valfilter, assoc
from aging.size_norm.data import AugmentationParams, TrainingPaths
from lightning.pytorch.loggers import CSVLogger, TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from aging.size_norm.lightning import SizeNormModel, ImageDisplay, BehaviorValidation

In [2]:
config = toml.load('/home/wg41/code/ontogeny/configs/00-sizenorm_training_template.toml')

In [3]:
paths = TrainingPaths(**dissoc(config['paths'], 'saving'))
checkpoint = None
save_folder = Path(config['paths']['saving'])
save_folder.mkdir(exist_ok=True, parents=True)

In [4]:
config['augmentation']['rng'] = random.Random(config['augmentation']['seed'])

In [5]:
pl.seed_everything(config['augmentation']['seed'])

[rank: 0] Global seed set to 0


0

In [6]:
augmentation = AugmentationParams(**dissoc(config['augmentation'], 'seed'))

In [7]:
match config['model']['lightning']['arch']:
    case 'ae':
        model_arch = models.Autoencoder
    case 'unet':
        model_arch = models.UNet
    case 'efficient':
        model_arch = models.EfficientAutoencoder
    case 'efficient_unet':
        model_arch = models.EfficientUNet
    case _:
        raise ValueError(f"Unknown model architecture {config['model']['arch']}")

In [8]:
model_params = merge(config['model'], config['model'][config['model']['lightning']['arch']])
model_params = valfilter(lambda x: not isinstance(x, dict), model_params)
if 'activation' in model_params:
    model_params = assoc(model_params, 'activation', getattr(torch.nn, model_params['activation']))

In [9]:
model = SizeNormModel(
    paths,
    augmentation,
    model=model_arch,
    model_params=model_params,
    batch_size=64,
    **dissoc(config['model']['lightning'], 'arch'),
    seed=config['augmentation']['seed']
)

In [10]:
ckpt_cb = ModelCheckpoint(
    save_folder,
    filename=f"{model_arch.__name__}" + "-{epoch:02d}-{val_loss:.2e}",
    monitor="val_loss",
    save_top_k=1,
    mode="min",
)

early_stopping_cb = EarlyStopping(
    monitor="val_loss",
    patience=config.get('callbacks', {}).get('stopping', {}).get('patience', 13),
    mode="min",
)

beh_val_cb = BehaviorValidation(
    paths.validation,
    log_interval=config.get('callbacks', {}).get('validation', {}).get('log_interval', 30),
    max_frames=config.get('callabacks', {}).get('validation', {}).get('max_frames', 8000),
)

In [11]:
image_cb = ImageDisplay()

trainer = pl.Trainer(
    max_epochs=config.get('trainer', {}).get('max_epochs', 85),
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    callbacks=[ckpt_cb, early_stopping_cb, beh_val_cb, image_cb],
    precision='16-mixed' if torch.cuda.is_available() else "bf16-mixed",
    logger=[CSVLogger(save_folder, name="size_norm_scan"), TensorBoardLogger(save_folder, name="size_norm_scan")],
)
trainer.fit(model, ckpt_path=checkpoint)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/wg41/miniconda3/envs/aging/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /n/groups/datta/win/longtogeny/size_norm/models/pre_final_model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                 | Params
-----------------------------------------------
0 | model | TopLevelTracedModule | 812 K 
-----------------------------------------------
812 K     Trainable params
0         Non-trainable params
812 K     Total params
3.249     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

/home/wg41/miniconda3/envs/aging/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/wg41/miniconda3/envs/aging/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x08\x00\x00\x00\x08\x08\x00\x00\x00\x00\xe1d\xe1…

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=201` reached.


In [12]:
model = SizeNormModel.load_from_checkpoint(ckpt_cb.best_model_path)

# save jit version of model
torch.jit.save(model.model, save_folder / "model.pt")